In [2]:
import ffmpeg
import subprocess
import json
import os
from pathlib import Path
from glob import glob

def get_chapters(mkv_file):
    """Extract chapter information from MKV file"""
    cmd = [
        'ffprobe',
        '-i', str(mkv_file),
        '-print_format', 'json',
        '-show_chapters',
        '-loglevel', 'error'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    data = json.loads(result.stdout)
    return data.get('chapters', [])

def filter_chapters_by_name(chapters, keyword):
    """Filter chapters that contain the keyword in their title"""
    matching_chapters = []
    
    for chapter in chapters:
        tags = chapter.get('tags', {})
        title = tags.get('title', '')
        
        if keyword.lower() in title.lower():
            matching_chapters.append({
                'title': title,
                'start': float(chapter['start_time']),
                'end': float(chapter['end_time'])
            })
    
    return matching_chapters

def extract_chapters_from_file(input_file, chapters, temp_dir='temp_segments'):
    """Extract matching chapters from a single file"""
    os.makedirs(temp_dir, exist_ok=True)
    
    temp_files = []
    base_name = Path(input_file).stem
    
    for i, chapter in enumerate(chapters):
        temp_output = os.path.join(temp_dir, f'{base_name}_segment_{i}.mkv')
        temp_files.append(temp_output)
        
        # Extract each chapter segment
        (
            ffmpeg
            .input(str(input_file), ss=chapter['start'], to=chapter['end'])
            .output(temp_output, c='copy')
            .overwrite_output()
            .run(quiet=True)
        )
        print(f"  Extracted: {chapter['title']} ({chapter['start']:.2f}s - {chapter['end']:.2f}s)")
    
    return temp_files

def merge_segments(segments, output_file, temp_dir='temp_segments'):
    """Merge multiple segments into a single output file"""
    if not segments:
        return False
    
    # Create concat file for ffmpeg
    concat_file = os.path.join(temp_dir, 'concat_list.txt')
    with open(concat_file, 'w') as f:
        for segment in segments:
            abs_path = os.path.abspath(segment)
            f.write(f"file '{abs_path}'\n")
    
    # Merge all segments
    subprocess.run([
        'ffmpeg',
        '-f', 'concat',
        '-safe', '0',
        '-i', concat_file,
        '-c', 'copy',
        output_file,
        '-y'
    ], check=True, capture_output=True)
    
    return True

def get_input_files(input_pattern):
    """Get input files from a glob pattern or list of files"""
    if isinstance(input_pattern, str):
        # It's a glob pattern
        files = sorted(glob(input_pattern))
        if not files:
            print(f"No files found matching pattern: {input_pattern}")
        return files
    elif isinstance(input_pattern, list):
        # It's already a list of files
        return input_pattern
    else:
        raise ValueError("input_pattern must be a string (glob pattern) or list of file paths")

def process_files(input_pattern, output, chapter_keyword, merge_all=True, output_dir=None, temp_dir='temp_segments'):
    """
    Process MKV files and extract matching chapters.
    
    Args:
        input_pattern: Glob pattern (str) or list of file paths
        output: Output filename (used when merge_all=True) or ignored when merge_all=False
        chapter_keyword: Keyword to filter chapters by title
        merge_all: If True, merge all chapters into a single file.
                   If False, create separate output files for each input file.
        output_dir: Directory for output files when merge_all=False.
                    Defaults to same directory as input files.
        temp_dir: Temporary directory for intermediate files
    """
    input_files = get_input_files(input_pattern)
    
    if not input_files:
        return
    
    print(f"Found {len(input_files)} input file(s)")
    
    all_segments = []
    file_segments = {}  # Track segments per input file
    
    # Process each input file
    for input_file in input_files:
        input_path = Path(input_file)
        
        # Check if file exists
        if not input_path.exists():
            print(f"\nWARNING: File not found, skipping: {input_file}")
            continue
        
        print(f"\nProcessing: {input_file}")
        
        # Get all chapters
        chapters = get_chapters(input_file)
        print(f"  Found {len(chapters)} total chapters")
        
        # Filter chapters by keyword
        matching_chapters = filter_chapters_by_name(chapters, chapter_keyword)
        print(f"  Found {len(matching_chapters)} chapters matching '{chapter_keyword}'")
        
        if matching_chapters:
            segments = extract_chapters_from_file(input_file, matching_chapters, temp_dir)
            all_segments.extend(segments)
            file_segments[input_file] = segments
        else:
            print(f"  No matching chapters found in {input_file}")
    
    if not all_segments:
        print("\nNo matching chapters found in any files!")
        return
    
    if merge_all:
        # Merge all segments into a single file
        print(f"\nTotal segments to merge: {len(all_segments)}")
        print("\nMerging all segments into single file...")
        
        if merge_segments(all_segments, output, temp_dir):
            print(f"\nFinal merged video saved to: {output}")
    else:
        # Create separate output files for each input file
        print(f"\nCreating separate output files...")
        
        for input_file, segments in file_segments.items():
            base_name = Path(input_file).stem
            
            # Determine output directory
            if output_dir:
                os.makedirs(output_dir, exist_ok=True)
                out_file = os.path.join(output_dir, f"{base_name}_filtered.mkv")
            else:
                out_file = str(Path(input_file).parent / f"{base_name}_filtered.mkv")
            
            if merge_segments(segments, out_file, temp_dir):
                print(f"  Saved: {out_file}")
    
    # Cleanup
    cleanup_temp_files(temp_dir)

def cleanup_temp_files(temp_dir):
    """Remove temporary directory and all its contents"""
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"\nCleaned up temporary files in {temp_dir}")

In [3]:
import re
# Example usage

chapter_keyword = "Episode"  # Keep only chapters with "Episode" in the name
start_episode = 35
end_episode = 37

# --- Merge all into single file ---
base = Path(r"C:\Torrent\TV\Attack on Titan\Season 2")
all_files = sorted(base.glob("*Shingeki no Kyojin*.mkv"))

# Extract episode number and filter
input_files = []
for f in all_files:
    match = re.search(r" - (\d+)", f.name)
    if match:
        ep_num = int(match.group(1))
        if start_episode <= ep_num <= end_episode:
            input_files.append(f)

process_files(
    input_files,
    output=rf"C:\Torrent\TV\Attack on Titan\Season 2\{start_episode}_{end_episode}.mkv",
    chapter_keyword=chapter_keyword,
    merge_all=True
)

# --- OR: Create separate files for each input ---
# process_files(
#     input_pattern,
#     output=None,  # Ignored when merge_all=False
#     chapter_keyword=chapter_keyword,
#     merge_all=False,
#     output_dir=r"C:\Torrent\TV\Attack on Titan\filtered"  # Optional, defaults to same dir as input
# )

Found 3 input file(s)

Processing: C:\Torrent\TV\Attack on Titan\Season 2\[MTBB] Shingeki no Kyojin - 35 (BD 1080p) [892FFD05].mkv
  Found 5 total chapters
  Found 1 chapters matching 'Episode'
  Extracted: Episode (172.02s - 1330.10s)

Processing: C:\Torrent\TV\Attack on Titan\Season 2\[MTBB] Shingeki no Kyojin - 36 (BD 1080p) [F178A12D].mkv
  Found 5 total chapters
  Found 1 chapters matching 'Episode'
  Extracted: Episode (155.00s - 1329.97s)

Processing: C:\Torrent\TV\Attack on Titan\Season 2\[MTBB] Shingeki no Kyojin - 37 (BD 1080p) [2C0C43E7].mkv
  Found 4 total chapters
  Found 1 chapters matching 'Episode'
  Extracted: Episode (126.98s - 1330.01s)

Total segments to merge: 3

Merging all segments into single file...

Final merged video saved to: C:\Torrent\TV\Attack on Titan\Season 2\35_37.mkv

Cleaned up temporary files in temp_segments
